In [1]:
BASE_PATH = "/tf/notebooks/CEIA/computer-vision-2/tp-final/models/yolov7"

import os

# !pip install -r {os.path.join(BASE_PATH, "yolov7/requirements.txt")}
# !apt-get update
# !apt-get install ffmpeg libsm6 libxext6 -y

import torch

In [2]:
PRETRAINED_PATH = os.path.join(BASE_PATH, "yolov7-pretrained")
WEIGHTS_FILE = os.path.join(PRETRAINED_PATH, "yolov7_training.pt")

if os.access(WEIGHTS_FILE, os.F_OK) is False:
    try:
        original_umask = os.umask(0)
        os.mkdir(PRETRAINED_PATH, mode=0o775)
    except FileExistsError:
        pass
    finally:
        os.umask(original_umask)

    # download COCO starting checkpoint
    !wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt -P {PRETRAINED_PATH}

In [3]:
RUNS_PATH = "/tf/notebooks/CEIA/computer-vision-2/tp-final/runs"
TRAIN_PROJECT_PATH = os.path.join(RUNS_PATH, "train")
TRAIN_SCRIPT = os.path.join(BASE_PATH, "yolov7/train.py")
DATASET_PATH = "/tf/datasets/final-dataset/yolov7-dataset"
DATA_YAML = os.path.join(DATASET_PATH, "data.yaml")

NUM_BATCHES = 4
NUM_EPOCHS = 50
FREEZE_LAYERS = 50 # backbone
IMG_SIZE = 640
experiment = "yolov7-50-epochs-640-2"

In [4]:
torch.cuda.empty_cache() 

!python {TRAIN_SCRIPT} \
--project {TRAIN_PROJECT_PATH} \
--name {experiment} \
--batch {NUM_BATCHES} \
--epochs {NUM_EPOCHS} \
--data {DATA_YAML} \
--img-size {IMG_SIZE} {IMG_SIZE} \
--weights {WEIGHTS_FILE} \
--device 0 \
--freeze {FREEZE_LAYERS}

In [5]:
# run tensorboard
# !tensorboard --logdir=/tf/notebooks/CEIA/computer-vision-2/tp-final/runs/train --host 0.0.0.0

In [6]:
TEST_PROJECT_PATH = os.path.join(TRAIN_PROJECT_PATH, f"{experiment}")
TEST_SCRIPT = os.path.join(BASE_PATH, "yolov7/test.py")
TEST_WEIGHTS = os.path.join(TRAIN_PROJECT_PATH, f"{experiment}/weights/best.pt")
TEST_DATA_PATH = os.path.join(DATASET_PATH, "test/images")
TEST_NAME = 'test'

CONFIDENCE_THRESHOLD = 0.7

!python {TEST_SCRIPT} \
 --project {TEST_PROJECT_PATH} \
 --name {TEST_NAME} \
 --batch {NUM_BATCHES} \
 --task test \
 --data {DATA_YAML} \
 --img-size {IMG_SIZE} \
 --weights {TEST_WEIGHTS} \
 --conf-thres {CONFIDENCE_THRESHOLD} \
 --device 0 \
 --verbose

Namespace(augment=False, batch_size=4, conf_thres=0.7, data='/tf/datasets/final-dataset/yolov7-dataset/data.yaml', device='0', exist_ok=False, img_size=640, iou_thres=0.65, name='test', no_trace=False, project='/tf/notebooks/CEIA/computer-vision-2/tp-final/runs/train/yolov7-50-epochs-640', save_conf=False, save_hybrid=False, save_json=False, save_txt=False, single_cls=False, task='test', v5_metric=False, verbose=True, weights=['/tf/notebooks/CEIA/computer-vision-2/tp-final/runs/train/yolov7-50-epochs-640/weights/best.pt'])
YOLOR 🚀 v0.1-112-g55b90e1 torch 1.12.1+cu102 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11177.25MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
/usr/local/lib/python3.8/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2895.)
  return _VF.meshgrid(tenso

In [7]:
# DETECT_SCRIPT = os.path.join(BASE_PATH, "yolov7/detect.py")
# DETECT_DATA_PATH = os.path.join(DATASET_PATH, "test/images")
# DETECT_NAME = 'detect'

# !python {DETECT_SCRIPT} \
#  --project {TEST_PROJECT_PATH} \
#  --name {DETECT_NAME} \
#  --weights {TEST_WEIGHTS} \
#  --conf-thres {CONFIDENCE_THRESHOLD} \
#  --source {DETECT_DATA_PATH}